Source:
* [Adventures in Machine Learning - Reinforcement learning tutorial using Python and Keras](https://adventuresinmachinelearning.com/reinforcement-learning-tutorial-python-keras/)

# Reinforcement learning with Keras

The input to the network is the one-hot encoded state vector. For instance, the vector which corresponds to state 1 is [0, 1, 0, 0, 0] and state 3 is [0, 0, 0, 1, 0]. In this case, a hidden layer of 10 nodes with sigmoid activation will be used. The output layer is a linear activated set of two nodes, corresponding to the two Q values assigned to each state to represent the two possible actions. Linear activation means that the output depends only on the linear summation of the inputs and the weights, with no additional function applied to that summation.

Building this network is easy in Keras

In [3]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense
# from keras.optimizers import Adam

Using TensorFlow backend.
/Users/admin/Projects/doggo/.venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/admin/Projects/doggo/.venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/admin/Projects/doggo/.venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/admin/Projects/d

In [4]:
model = Sequential()
model.add(InputLayer(batch_input_shape=(1, 5)))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (1, 10)                   60        
_________________________________________________________________
dense_2 (Dense)              (1, 2)                    22        
Total params: 82
Trainable params: 82
Non-trainable params: 0
_________________________________________________________________
None


First, the model is created using the Keras Sequential API. Then an input layer is added which takes inputs corresponding to the one-hot encoded state vectors. Then the sigmoid activated hidden layer with 10 nodes is added, followed by the linear activated output layer which will yield the Q values for each action. Finally the model is compiled using a mean-squared error loss function (to correspond with the loss function defined previously) with the Adam optimizer being used in its default Keras state.

To use this model in the training environment, the following code is run which is similar to the previous -greedy Q learning methodology with an explicit Q table:

In [ ]:
# now execute the q learning
y = 0.95
eps = 0.5
decay_factor = 0.999

r_avg_list = []

for i in range(num_episodes):
    
    s = env.reset()
    
    eps *= decay_factor
    
    if i % 100 == 0:
        print("Episode {} of {}".format(i + 1, num_episodes))
    
    done = False
    r_sum = 0
    
    while not done:
        
        if np.random.random() < eps:
            a = np.random.randint(0, 2)
        else:
            a = np.argmax(model.predict(np.identity(5)[s:s + 1]))
        
        new_s, r, done, _ = env.step(a)
        
        target = r + y * np.max(model.predict(np.identity(5)[new_s:new_s + 1]))
        target_vec = model.predict(np.identity(5)[s:s + 1])[0]
        target_vec[a] = target
        model.fit(np.identity(5)[s:s + 1], target_vec.reshape(-1, 2), epochs=1, verbose=0)
        
        s = new_s
        r_sum += r
    
    r_avg_list.append(r_sum / 1000)

The first major difference in the Keras implementation is the following code:

In [ ]:
# if np.random.random() < eps:
#     a = np.random.randint(0, 2)
# else:
#     a = np.argmax(model.predict(np.identity(5)[s:s + 1]))

The first condition in the if statement is the implementation of the $\epsilon$-greedy action selection policy that has been discussed already. The second condition uses the Keras model to produce the two Q values – one for each possible state. It does this by calling the model.predict() function. Here the numpy identity function is used, with vector slicing, to produce the one-hot encoding of the current state s. The standard numpy argmax function is used to select the action with the highest Q value returned from the Keras model prediction.

The second major difference is the following four lines:

In [ ]:
# target = r + y * np.max(model.predict(np.identity(5)[new_s:new_s + 1]))
# target_vec = model.predict(np.identity(5)[s:s + 1])[0]
# target_vec[a] = target
# model.fit(np.identity(5)[s:s + 1], target_vec.reshape(-1, 2), epochs=1, verbose=0)

The first line sets the target as the Q learning updating rule that has been previously presented. It is the reward r plus the discounted maximum of the predicted Q values for the new state, new_s. This is the value that we want the Keras model to learn to predict for state s and action a i.e. Q(s,a). However, our Keras model has an output for each of the two actions – we don’t want to alter the value for the other action, only the action a which has been chosen. So on the next line, target_vec is created which extracts both predicted Q values for state s. On the following line, only the Q value corresponding to the action a is changed to target – the other action’s Q value is left untouched.

The final line is where the Keras model is updated in a single training step. The first argument is the current state – i.e. the one-hot encoded input to the model. The second is our target vector which is reshaped to make it have the required dimensions of (1, 2). The third argument tells the fit function that we only want to train for a single iteration and finally the verbose flag simply tells Keras not to print out the training progress.

Running this training over 1000 game episodes reveals the following average reward for each step in the game:

As can be observed, the average reward per step in the game increases over each game episode, showing that the Keras model is learning well (if a little slowly).

We can also run the following code to get an output of the Q values for each of the states – this is basically getting the Keras model to reproduce our explicit Q table that was generated in previous methods:

State 0 – action [[62.734287 61.350456]]

State 1 – action [[66.317955 62.27209 ]]

State 2 – action [[70.82501 63.262383]]

State 3 – action [[76.63797 64.75874]]

State 4 – action [[84.51073 66.499725]]

This output looks sensible – we can see that the Q values for each state will favor choosing action 0 (moving forward) to shoot for those big, repeated rewards in state 4. Intuitively, this seems like the best strategy.